# What app should i build?

This projects is aimed to help decide what kind of app I should build with intent on making a profitable app. As I only know 2 languages (swedish and english), this will project will focus on english apps only.

The first part of this project will focus on free apps (completed) and the second part will focus on paid apps (not yet completed). This is because the different payment models most likely will affect what type of genres/apps will be a viable option.

Data for this project is from the following sources:

https://www.kaggle.com/lava18/google-play-store-apps/home

https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home

# 1. Importing and Exploring Data

In [26]:
#reading datasets from googleplay and ios
from csv import reader

##Google play
opened_file = open('googleplaystore.csv', encoding="utf8")
read_file = reader(opened_file)
googleplay = list(read_file, )
googleplay_header = googleplay[0]
googleplay = googleplay[1:]

### The App Store data set ###
opened_file = open('AppleStore.csv', encoding="utf8")
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In [27]:
#explore data
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line between rows
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print(googleplay_header)
print('\n')
explore_data(googleplay, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


The Google Play dataset has 10841 apps and 13 columns. Relevant columns for analyzing profitability would likely be: App, Category, Reviews, Installs, Price, Genres. Rating, Type, content rating might be interesting but might also have high correlation with other columns: Type and Category might show autocorrelation if both are chosen for example. Last updated, Current ver, android ver not deemed interesting for this analysis.

In [28]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


The iOS dataset has 7197 rows and 16 columns. Relevant columns will be track_name, currency, price, rating_count_tot, rating_count_ver and prime_genre. Not all columns are self explanatory though and require a google check to understand.

In [29]:
#clean data - Deleting wrong data
#according to kaggle discussion forum the row 10472 in the googleplay dataset
#will be incorrect
print(googleplay_header)
print('\n')
print(googleplay[10472])
#This is true since the rating is '19' when max value is 5.


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [30]:
#ONLY RUN THIS ONCE OR YOU WILL DELETE WRONG ROWS.
del googleplay[10472]

In [31]:
#Cleaning data - removing multiple entries
#there are multiple duplicates in the data. This will be adjusted by creating a dictionary where each key is the unique app name
#For duplicate app entries, older entries will be removed and will be judged by number of reviews.
duplicate_apps = []
unique_apps = []

for app in googleplay:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Amount of duplicate apps in dataset:', len(duplicate_apps))

Amount of duplicate apps in dataset: 1181


In [32]:
#Cleaning data - removing multiple entries
#Building the dictionary

reviews_max = {}
for app in googleplay:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
#The length of the dictionary should be the length of googleplay data minus duplicate apps
if (len(googleplay)-len(reviews_max)) == len(duplicate_apps):
    print("Length checks out!")
else:
    print("Ohoh, somethings wrong here")
        

Length checks out!


In [33]:
googleplay_clean = []
already_added = []

for app in googleplay:
    name = app[0]
    n_reviews = float(app[3])
    
    if reviews_max[name] == n_reviews and name not in already_added:
        googleplay_clean.append(app)
        already_added.append(name) # make sure this is inside the if block

In [34]:
if (len(googleplay_clean)) == len(reviews_max):
    print("Ok!")
else:
    print("no bueno")

Ok!


# 2. Remove non-english apps

Now we will remove all Non-english apps from the dataset. The problem is how to do it. In theory we could delete all characters outside of the english Alphabet A-Z, but this will delete english apps which might contain characters in it's title that are not contained in the english dictionary which is problematic.

An easy (alebit non-optimised) way to do this is too limit the amount of non-ascii charcters allowed in the title to 2 or 3. This might let some non english apps slip, but will work OK in the meanwhile.

In [35]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

In [36]:
googleplay_english = []
ios_english = []

for app in googleplay_clean:
    name = app[0]
    if is_english(name):
        googleplay_english.append(app)
        
for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

# 3a. Isolating the Free apps
Since we are only looking at the free apps, we now need to isolate this category from the paid apps in the data set since we are interested in in-app ads as a source of revenue. This will be the last part of the data cleaning process.

In [37]:
googleplay_free = []
ios_free = []

for app in googleplay_english:
    price = app[7]
    if price == "0":
        googleplay_free.append(app)

for app in ios_english:
    price = app[4]
    if price == "0.0":
        ios_free.append(app)
    

# 4a - Analysis 1, Most popular Genres
The main objective is to understand what kind of apps are more likey to gain users. This is because more users = higher revenue from ads.

In [38]:
def freq_table(dataset,index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key]/total)*100
        table_percentages[key] = percentage
    
    return table_percentages
    
    
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])    

In [39]:
#analyse prime_genre column for iOS
display_table(ios_free, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Most common genre for the iOS platform is Games (58%!) followed by entertainment and Photo & Video. The general impression is that most apps are focused toward entertainment purposes. Large number of apps does not mean large number of users, but there is probably a correlation.

In [40]:
#analyse Category columns for google play
display_table(googleplay_free, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

On the google play platform the largest category is family with 18% followed by games 9.72% and tools 8.46%. This might be caused by different categorization on the two platforms: Family might include family oriented games for instance. It might also be that google play has a larger library or that it has a different clientel. Even so, it seems that less apps on google store are for entertainment purposes.

The reason why we look at category rather than genre for google play is because the caregory column is much more granular than category for googleplay or genre for iOS which are much closer comparison.

# 5a - Analysis, Most popular apps

To find the most popular apps on the appstore we will calculate the average number of installs for each app genre. Since there is an inherit problem in thinking that the most available apps is the same as the most popular apps, this might help circumvent that problem.

For google play we use the column "Installs", for iOS we use the column "rating_count_tot" since installs is not available, but you can only give 1 rating per install.

In [41]:
#iOS
genres_ios = freq_table(ios_free, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_free:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Turns out that on iOS the most popular apps are Navigation, followed by Social Networking and Reference. Or atleast, those are the ones that are most popular to review... This might also be skewed by few VERY popular apps. Google maps might have a very high install rate for example which might skew the entire dataset.

In [42]:
for app in ios_free:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Unsurprisingly, the top 3 Navigation apps show a heavy skew. Although Waze turns out to be more popular than google maps which is a surprise! Most likely Social Networking show the same skew (facebook, snapchat etc). The same is most likely for many genres where few apps have a disproportianate amount of installs meaning that just because a genre has many installs doesnt mean it necessarily will be a profitable genre to build an app around.

A couple genres stand out with not having quite the same skew in apps (although skew still exists): Reference, book, weather, food and drink and finance. Since iOS is skewed toward entertainment apps, this part of the market might be heavily saturated.

Weather apps: people dont spend a lot of time in app so getting in app revenue is hard as is getting a weather feed, food and drink is generelly apps made by companies such as mcdonalds which would require some kind of restaurant or food service behind the app. Finance apps require a lot of domain knowledge. These apps are most likely less "good" if you want to build a free app and require a paid version to be able to make a profit.

Book and Reference categories also seem to skew toward big apps, but also have several smaller apps. The upside with a book app is low cost. Maybe apps with quotes, quizzes or audio can tempt users to log in daily. Generally it seems like a lot of genres are saturated.

In [43]:
for app in ios_free:
    if app[-5] == 'Book':
        print(app[1], ':', app[5])
print('\n')

for app in ios_free:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])


Kindle – Read eBooks, Magazines & Textbooks : 252076
Audible – audio books, original series & podcasts : 105274
Color Therapy Adult Coloring Book for Adults : 84062
OverDrive – Library eBooks and Audiobooks : 65450
HOOKED - Chat Stories : 47829
BookShout: Read eBooks & Track Your Reading Goals : 879
Dr. Seuss Treasury — 50 best kids books : 451
Green Riding Hood : 392
Weirdwood Manor : 197
MangaZERO - comic reader : 9
ikouhoushi : 0
MangaTiara - love comic reader : 0
謎解き : 0
謎解き2016 : 0


Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tool

In [44]:
#GooglePlay
display_table(googleplay_free, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


Contrary to the iOS data, there is data on the number of installations on the GooglePlay plattform, allthough they are very rough categories. 100 000+ can be anywhere from 100 000 < x < 1 000 000, which is a bit of a problem.

In [45]:
categories_googleplay = freq_table(googleplay_free, 1)

for category in categories_googleplay:
    total = 0
    len_category = 0
    for app in googleplay_free:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

Communications have the most installs, which ofc will be skewed by few apps like facebook, messenger, whatsapp, gmail etc.

In [46]:
for app in googleplay_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

Similiarly all categories no matter if its social, photo, etc will be skewed by large apps which will make the categories seem more popular than they really are. But if we look at books and reference we see a smaller skew (like iOS). This is because it includes many niche apps that dont require things like network effects. Books on programming for example! Only very few have many installs.

In [47]:
for app in googleplay_free:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

# 6a - Conclusion of Free Apps
The market for free apps are both saturated and also heavily skewed toward apps that already have a big following (and also have network effects). This means building a free app which can generate a sustainable profit is hard. One possibility is building apps in the reference/books category which can build upon smaller niches such as programming. The idea would not be to write your own book, but rather build apps which are based on derivatives of the books such as summarys, quotes or similar.

